In [2]:
import tensorflow as tf

In [3]:
import os
os.chdir("../")

In [4]:

model = tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    params_image_size: list
    params_batch_size: int
    mlflow_uri: str
    all_params: dict

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/kumarsunny30066/DeepCNNClassifier.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="kumarsunny30066"
os.environ["MLFLOW_TRACKING_PASSWORD"]="a79f1b0d67e5365a47d58ceb8de8e444b0331ac6"

In [16]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/PetImages",
            mlflow_uri="https://dagshub.com/kumarsunny30066/DeepCNNClassifier.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [17]:
import tensorflow as tf
from pathlib import Path
from urllib.parse import urlparse 
import mlflow
import mlflow.keras

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.025
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [18]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()
    
except Exception as e:
   raise e

[2022-09-29 17:30:47,191: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-09-29 17:30:47,199: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-29 17:30:47,204: INFO: common]: created directory at: artifacts
Found 624 images belonging to 2 classes.
20/20 [==============================] - 222s 11s/step - loss: 3.1746 - accuracy: 0.7516
[2022-09-29 17:34:31,045: INFO: common]: json file saved at: scores.json
[2022-09-29 17:34:36,096: WARNING: save]: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.
INFO:tensorflow:Assets written to: C:\Users\aniyant\AppData\Local\Temp\tmpzq7_1syp\model\data\model\assets
[2022-09-29 17:34:38,603: INFO: builder_impl]: Assets written to: C:\Users\aniyant\AppData\Local\Temp\tmpzq7_1syp\model\data

Successfully registered model 'VGG16Model'.
2022/09/29 17:42:03 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
